In [1]:
import pandas as pd

# Limpieza

In [2]:
df = pd.read_csv("datasets/paradas-de-colectivo.csv", delimiter=";", encoding="latin1")

In [3]:
df.head(5)

,X,Y,STOP_ID,TIPO,CALLE,NUMERO,ENTRE1,ENTRE2,LINEAS,DIR_NORM,CALLE_NORM,ALTURA_NOR,COORDX,COORDY,METROBUS,STOP_NAME,STOP_DESC,FUENTE,VERIFICADA,FECHA_ULTI
0,"-58,4327908137487","-34,5923809470",1000086,PARADAS CON REFUGIO,CORDOBA AV.,4859,ARMENIA,GURRUCHAGA,"{151,168}",CORDOBA AV. 4859,CORDOBA AV.,4859.0,102.798.829.931.999.000.000,104.091.825.983.000.000.000,F,NaN,NaN,RELEVAMIENTO2015,T,9/4/2015
1,"-58,5140298647844","-34,6016371880",1005465,PARADA NO OFICIAL,NUEVA YORK,4224,NaN,GUALEGUAYCHU,{105},NUEVA YORK 4224,NUEVA YORK,4224.0,95.346.600.539.799.900.000,103.064.210.534.000.000.000,F,NaN,NaN,RELEVAMIENTO2015,T,17/4/2015
2,"-58,4497660921390","-34,5544664433",1002096,PARADAS CON REFUGIO,DEL LIBERTADOR AV.,6250,BLANCO ENCALADA,OLAZABAL,"{15,29}",DEL LIBERTADOR AV. 6250,DEL LIBERTADOR AV.,6250.0,101.242.081.519.000.000.000,108.298.117.206.000.000.000,F,NaN,NaN,RELEVAMIENTO2015,T,10/4/2015
3,"-58,4941044948861","-34,5623407465",1006607,PARADAS CON REFUGIO,"LARRALDE, CRISOLOGO AV.",5187,NaN,NaN,"{110,71}","LARRALDE, CRISOLOGO AV. 5187","LARRALDE, CRISOLOGO AV.",5187.0,97.172.965.995.399.900.000,107.424.229.065.000.000.000,F,NaN,NaN,RELEVAMIENTO2015,T,14/4/2015
4,"-58,4844741311517","-34,5757583663",1004767,PARADAS DE POSTE,OLAZABAL AV.,4920,NaN,TRIUNVIRATO AV.,{90},OLAZABAL AV. 4920,OLAZABAL AV.,4920.0,98.057.080.321.899.900.000,105.936.009.144.000.000.000,F,NaN,NaN,RELEVAMIENTO2015,T,14/4/2015


Reemplazamos las ',' por '.' en X e y para que sean coordenadas válidas. 

In [4]:
df.X = df.X.str.replace(",",".")
df.Y = df.Y.str.replace(",",".")

Eliminamos las que no tienen calle o altura. **Chequear si esto está bien**.

In [5]:
df = df[~df.CALLE.isnull()]
df = df[df.NUMERO != 'S/N']

Creamos una nueva tabla donde cada fila es la correspondencia entre una parada y una línea que pasar por allí.

In [6]:
filas = []

# Separa cada parada en una fila nueva por línea que pasa por allí.
def separar_fila(fila, filas_nuevas):
    lineas = fila.LINEAS[1:-1].split(',')
    for linea in lineas:
        fila_nueva = {'STOP_ID': fila.STOP_ID, 'LINEA': linea}
        filas_nuevas.append(fila_nueva)

df.apply(separar_fila, axis=1, args=(filas,))
lineas_por_parada = pd.DataFrame(filas)

Por el momento exportamos solo los datos necesarios para el punto 1. Esto incluye el id de la parada, las coordenadas, la calle y la altura. Luego iremos limpiando y agregando los otros campos a medida que sean necesarios. 

In [7]:
df[['STOP_ID','X','Y', 'CALLE', 'NUMERO' ]].to_csv('datasets/paradas-de-colectivo-clean.csv', index=False)
lineas_por_parada.to_csv("datasets/lineas_por_parada.csv", index = False)

In [20]:
df_clean = pd.read_csv("datasets/paradas-de-colectivo-clean.csv", delimiter=",", encoding="latin1")

In [21]:
df_clean.head(5)

,STOP_ID,X,Y,CALLE,NUMERO
0,1000086,-58.432791,-34.592381,CORDOBA AV.,4859
1,1005465,-58.514030,-34.601637,NUEVA YORK,4224
2,1002096,-58.449766,-34.554466,DEL LIBERTADOR AV.,6250
3,1006607,-58.494104,-34.562341,"LARRALDE, CRISOLOGO AV.",5187
4,1004767,-58.484474,-34.575758,OLAZABAL AV.,4920


## Clean up adicional para otros campos

Convertimos en bool el metrobus

In [8]:
df.METROBUS = df.METROBUS == 'T'

Convertimos en bool la verificación.

In [9]:
df.VERIFICADA = df.VERIFICADA == 'T'

# Exploración

In [10]:
[x  for x in df.columns]

['X',
 'Y',
 'STOP_ID',
 'TIPO',
 'CALLE',
 'NUMERO',
 'ENTRE1',
 'ENTRE2',
 'LINEAS',
 'DIR_NORM',
 'CALLE_NORM',
 'ALTURA_NOR',
 'COORDX',
 'COORDY',
 'METROBUS',
 'STOP_NAME',
 'STOP_DESC',
 'FUENTE',
 'VERIFICADA',
 'FECHA_ULTI']

In [11]:
df.dtypes

X              object
Y              object
STOP_ID         int64
TIPO           object
CALLE          object
NUMERO         object
ENTRE1         object
ENTRE2         object
LINEAS         object
DIR_NORM       object
CALLE_NORM     object
ALTURA_NOR    float64
COORDX         object
COORDY         object
METROBUS         bool
STOP_NAME      object
STOP_DESC      object
FUENTE         object
VERIFICADA       bool
FECHA_ULTI     object
dtype: object

In [12]:
df.TIPO.unique()

array(['PARADAS CON REFUGIO', 'PARADA NO OFICIAL', 'PARADAS DE POSTE'],
      dtype=object)

In [13]:
df.CALLE.head().unique()

array(['CORDOBA AV.', 'NUEVA YORK', 'DEL LIBERTADOR AV.',
       'LARRALDE, CRISOLOGO AV.', 'OLAZABAL AV.'], dtype=object)

In [14]:
df.NUMERO.hasnans

False

In [15]:
df.LINEAS.head()

0    {151,168}
1        {105}
2      {15,29}
3     {110,71}
4         {90}
Name: LINEAS, dtype: object

`ALTURA_NOR` debería ser int, pero tiene NaNs y eso le causa problemas a pandas.

In [16]:
df.ALTURA_NOR.hasnans

True

In [17]:
df.COORDX.head()

0    102.798.829.931.999.000.000
1     95.346.600.539.799.900.000
2    101.242.081.519.000.000.000
3     97.172.965.995.399.900.000
4     98.057.080.321.899.900.000
Name: COORDX, dtype: object

In [62]:
df.STOP_DESC.unique()

array([nan, 'CABILDO AV.&&RAMALLO', 'DEL LIBERTADOR AV.&&GODOY CRUZ', ...,
       'ROCA, CNEL. AV.&&SALADILLO', '27 DE FEBRERO AV.&&PAZ, GRAL. AV.',
       '27 DE FEBRERO AV.'], dtype=object)

In [64]:
df.FUENTE.unique()

array(['RELEVAMIENTO2015', 'USIG'], dtype=object)

In [1]:
df.FECHA_ULTI.astype?

Object `df.FECHA_ULTI.astype` not found.
